In [ ]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
env = gym.make("LunarLander-v2", render_mode = 'rgb_array')

In [ ]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [ ]:
class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen = capacity)
    
    def push(self, *args):
        self.memory.append(Transition(*args))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128,128)
        self.layer3 = nn.Linear(128, n_actions)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.99
EPSILON = 0.9
MIN_EPSILON = 0.05
EPSILON_DECAY = 1000
TAU = 0.005
LR = 1e-4

In [ ]:
n_actions = env.action_space.n
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr = LR, amsgrad = True)
memory = ReplayMemory(10000)

steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    epsilon_threshold = MIN_EPSILON + (EPSILON - MIN_EPSILON) * math.exp(-1.*steps_done/EPSILON_DECAY)
    steps_done += 1
    if sample > epsilon_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1,1)
    else:
        return torch.tensor([[env.action_space.sample()]], device = device, dtype = torch.long)

episode_durations = []

def plot_durations(show_result = False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype = torch.float)
    if show_result:
        plt.title('Results')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())
    
    plt.pause(0.001)
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait = True)
        else:
            display.display(plt.gcf())

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device = device, dtype = torch.bool)
    
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    next_state_values = torch.zeros(BATCH_SIZE, device = device)
    
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    
    expected_state_action_values = reward_batch + (next_state_values * GAMMA)
    
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
def show_game(num_games = 3):
    for i in range(3):
        total_reward = 0
        state = env.reset()
        img = plt.imshow(env.render())
        for j in range(500):
            action = select_action(state)
            img.set_data(env.render())
            plt.axis('off')
            display.display(plt.gcf())
            display.clear_output(wait = True)
            state, reward, terminated, truncated, _ = env.step(action.item())
            done = terminated or truncated
            state = torch.tensor(state, dtype = torch.float32, device = device).unsqueeze(0)
            total_reward += reward
            if done:
                print(f"Game: {i} | Total_reward: {total_reward} | Game_Lenght: {j}\n")
                break


In [ ]:
# show_game()

In [ ]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    state, info = env.reset()
    state = torch.tensor(state, dtype = torch.float32, device = device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device = device)
        done = terminated or truncated
        
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype = torch.float32, device = device).unsqueeze(0)
        memory.push(state, action, next_state, reward)
        
        state = next_state
        
        optimize_model()
        
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        
        if done:
            episode_durations.append(t+1)
            plot_durations(show_result=False)
            if i_episode %5 == 0 and True:
                try:
                    show_game(num_games = 1)
                except:
                    print('Game_Show_Error!!!...')
                    pass
                print("Continuing")
            break
print('Complete')
plot_durations(show_result = False)
plt.ioff()
plt.show()

In [ ]:
show_game()

In [ ]:
def do_simulations(num_games = 3):
    for i in range(3):
        total_reward = 0
        state = env.reset()
        img = plt.imshow(env.render())
        for j in range(500):
            action = select_action(state)
            img.set_data(env.render())
            plt.axis('off')
            display.display(plt.gcf())
            display.clear_output(wait = True)
            state, reward, terminated, truncated, _ = env.step(action.item())
            done = terminated or truncated
            state = torch.tensor(state, dtype = torch.float32, device = device).unsqueeze(0)
            total_reward += reward
            if done:
                print(f"Game: {i} | Total_reward: {total_reward} | Game_Lenght: {j}\n")
                break


In [ ]:
plot_durations(show_result=False)

In [ ]:
from pathlib import Path

MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents = True, exist_ok = True)

MODEL_NAME = 'DQN_LunarLander_v0.pth'
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj = policy_net.state_dict(), f = MODEL_SAVE_PATH)